# Eigenvalue solvers in NGSolve

The goal of this section is to solve (generalized) matrix eigenvalue problems to find $u\in\mathbb C^N$, $\lambda\in\mathbb C$ such that

$$
A u = \lambda M u
$$
for given matrices $A,M\in\mathbb C^{N\times N}$

## Shift-and-invert

Algorithms like the power iteration or the Arnoldi method approximate eigenvectors corresponding to the largest magnitude eigenvalues. However in wave applications typically the smallest magnitude eigenvalues or ones in a specific region are sought.

The following observation transforms the generalized eigenvalue problem into a problem with the same eigenvectors but eigenvalues sorted by closeness to a given shift:

Let $\lambda_0\in \mathbb C$ be not an eigenvalue. Then the problem to find $u\in \mathbb C^N$, $\tilde\lambda\in\mathbb C$ such that

$$
(A-\lambda_0M)^{-1}Mu = \tilde\lambda u
$$

has the same eigenvectors as the original problem with the eigenvalues given by 
$$
\tilde \lambda = \frac{1}{\lambda-\lambda_0}.
$$

Thus the largest magnitude eigenvalues $\tilde \lambda$ of the shifted problem correspond to the ones of the original problem closest to the shift $\lambda_0$.

## Arnoldi algorithm

The idea of the arnoldi algorithm is similiar to the one of the power iteration: for a given starting vector $v_0$ the Krylow space of order $k$ is defined by 
$$V_k:=\mathrm{span}\{v_0,v_1=M^{-1}A v_0,\ldots,v_k=(M^{-1}A)^kv_0\}$$  

After a given number of steps the problem (i.e., the large matrices) is projected onto the (small) Krylow space and the small eigenvalue problem is solved. The resulting approximations are approximations to the largest magnitude eigenvalues.

## Shift-and-invert Arnoldi algorithm

It is common to combine these two ideas.
Suppose we want to compute the eigenvalues of the Laplacian on the `unit_square`. Then the matrices in `NGSolve` are assembled as usual

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

In [42]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

fes = H1(mesh,order = 4,complex=True)

u,v = fes.TnT()

A = BilinearForm(grad(u)*grad(v)*dx).Assemble().mat
M = BilinearForm(u*v*dx).Assemble().mat

To use `ArnoldiSolver` we have to supply a `multidim` `GridFunction` to store the eigenvectors.
The actual Krylow space is twice as large as the dimension of the `GridFunction`, however half of the approximated eigenvalues are thrown away (rule of thumb)

In [43]:
gfu = GridFunction(fes,multidim=100)
shift = 1

lam = ArnoldiSolver(A,M, shift=shift, vecs = list(gfu.vecs),freedofs=fes.FreeDofs())

In [44]:
import numpy as np
print(np.array(lam)/np.pi**2)

[-2.69973866e-16-8.91179602e-19j  1.00000000e+00+1.07853344e-15j
  1.00000000e+00-1.15458387e-15j  2.00000000e+00-8.33069060e-17j
  4.00000000e+00-1.49634854e-15j  4.00000000e+00-1.79317854e-15j
  5.00000001e+00+3.25555212e-15j  5.00000001e+00-1.48475067e-16j
  8.00000006e+00+3.32855662e-15j  9.00000009e+00-2.42333473e-14j
  9.00000016e+00-2.50028046e-14j  1.00000002e+01+5.69862784e-14j
  1.00000002e+01+2.35984423e-15j  1.30000007e+01+2.21310326e-14j
  1.30000009e+01+3.52467275e-15j  1.60000028e+01+7.22712190e-14j
  1.60000015e+01-1.11431478e-14j  1.70000021e+01-1.26965772e-13j
  1.70000035e+01+3.69521069e-14j  1.80000042e+01-2.52617645e-15j
  2.00000056e+01-6.30782147e-14j  2.00000073e+01-9.90543694e-16j
  2.60000162e+01-1.23450891e-13j  2.60000288e+01-2.96067105e-14j
  2.50000277e+01-3.15106383e-14j  2.50000142e+01+1.30603161e-13j
  2.50000220e+01+4.31229212e-14j  2.50000187e+01-6.62194628e-14j
  2.90000474e+01-6.69627443e-14j  2.90000339e+01-4.95366957e-14j
  3.20000788e+01-5.787408

In [45]:
Draw(gfu)

WebGuiWidget(value={'gui_settings': {}, 'ngsolve_version': '6.2.2403', 'mesh_dim': 2, 'order2d': 2, 'order3d':…

BaseWebGuiScene

## Explicit methods

Explicit eigenvalue solvers in `NGSolve` include `PINVIT` and `LOBPCG`

In [14]:
from ngsolve.solvers import *

In [16]:
help(PINVIT)
help(LOBPCG)

Help on function PINVIT in module ngsolve.eigenvalues:

PINVIT(mata, matm, pre, num=1, maxit=20, printrates=True, GramSchmidt=True)
    preconditioned inverse iteration

Help on function LOBPCG in module ngsolve.eigenvalues:

LOBPCG(mata, matm, pre, num=1, maxit=20, initial=None, printrates=True, largest=False)
    Knyazev's cg-like extension of PINVIT



In [32]:
lam,vecs = PINVIT(A,M,M.Inverse(),num=4,maxit=10000,printrates=False)
print(np.array(lam)/np.pi**2)


[6.34976576e-08 1.00003153e+00 1.04891865e+00 3.97291471e+00]


In [31]:
lam,vecs = LOBPCG(A,M,M.Inverse(),num=4,maxit=1000,printrates=False)
print(np.array(lam)/np.pi**2)


[-1.58926725e-16  1.00000000e+00  1.00000000e+00  2.00000000e+00]
